# 实验记录：数据集探索与 pdays 特征分析
**日期：** 2025-12-26  
**主要任务：** 数据初步统计分布查看、解决环境兼容性问题。

## 1. 环境配置记录
- **问题：** 安装了 NumPy 2.x 导致 pandas/matplotlib 崩溃。
- **解决：** 在 `pyproject.toml` 中锁定 `"numpy<2"`，使用 `uv sync` 重新构建虚拟环境。

## 2. 数据分布探索 (pdays 字段)
### 2.1 原始全量分布
![原始分布](plots/distribution_of_pdays_only_normal_customers.png)
- **观察：** 数据在 949 附近有极大比例的堆积。
- **结论：** 949 代表“从未联系过”，属于哨兵值。

### 2.2 过滤特殊值后的分布
![过滤后的分布](plots/distribution_of_pdays.png)
- **操作：** 排除 `pdays == 999` 的记录。
- **观察：** 发现 0-949 天之间的分布非常平坦。
- **结论：** 可以终点考虑一下0-30天之间，然后把30-949天设为一种时间段，再把949天之后设为一个时间段。



基于 EDA 探索，我们将 `pdays` 字段从连续数值转换为三个定性区间，以消除哨兵值（如 949+）对逻辑回归权重的干扰：

(plots/the purchasing_rate of different pdays.png)
| 类别名称 | pdays 范围 | 业务定义 |
| :--- | :--- | :--- |
| **pdays_active** | $0 \le x \le 7$ | **高频互动区**：近期刚联系，转化意愿通常最强。 |
| **pdays_followup** | $7 < x \le 20$ | **低频维护区**：有一定记忆，但热度开始消退。 |
| **pdays_cold_new** | $x > 20$ | **冷淡/新客区**：包括远期联系及从未联系的群体（含 949+ 堆积点）。 |

再对这些对应的数据进行one-hot 编码
- [x] **优化：哑变量陷阱处理**
    - **操作：** 在 `pd.get_dummies` 中添加 `drop_first=True`。
    - **原因：** 逻辑回归对多重共线性敏感，移除一个基准类（Base Category）能确保模型矩阵可逆，使 Baseline 的系数估计更稳定。


- [x] **发现：Duration 对数化后的双峰效应**
    - **现象**：分布图呈现明显的双峰特征（Bimodal）。
    - **解释**：左峰代表“快速拒绝型”短通话，右峰代表“深度兴趣型”长通话。
    - **价值**：模型可以利用这两个峰所在的区间，非常容易地切分出“无意愿”和“潜在意愿”客户，这是 Baseline 能获得高 Accuracy 的关键原因。

# 机器学习实验日志：关于类别权重（Class Weight）与评估指标的深度复盘

## 1. 实验观察
* **实验 A (Baseline)**: 使用 `GradientBoostingClassifier` 或默认 `LogisticRegression`，准确率 **~93%**。
* **实验 B (新设计)**: 在 `model_configs.py` 中设置 `class_weight='balanced'`，准确率骤降至 **~79%**。

## 2. 核心发现：准确率陷阱 (The Accuracy Trap)

### 为什么准确率会从 93% 掉到 79%？
1. **数据不平衡性**: 银行营销数据集（Bank Marketing）中，“不订阅 (no)”的用户占绝大多数（约 85%-90%）。
2. **模型的“投机”行为**: 
   - 当不设置 `class_weight` 时，模型发现只需盲目预测“no”就能获得极高的准确率。
   - 这种 93% 的准确率往往伴随着极低的 **召回率 (Recall)**，即模型几乎找不出真正的订阅客户。
3. **强制平衡的影响**: 
   - 设置 `class_weight='balanced'` 后，算法强制加大了对“yes”样本猜错的惩罚。
   - 模型为了捕捉那 10% 的潜在客户，变得更加“激进”，导致将大量“no”错误预测为“yes”（误报增加）。
   - **结果**: 总准确率（Accuracy）下降，但对业务真正有用的“yes”识别能力提升了。

## 3. 关键结论对比

| 指标 | 未开启 Balanced (93%) | 开启 Balanced (79%) |
| :--- | :--- | :--- |
| **准确率 (Accuracy)** | 极高 (但可能具有误导性) | 较低 |
| **预测风格** | 保守、倾向于预测“不购买” | 敏感、倾向于挖掘潜在客户 |

## 4. 后续优化动作 (Action Items)(gemini 建议但是如何实现)
1. **指标切换**: 停止将 Accuracy 作为唯一评价标准。后续应引入 **F1-Score**、**AUC-ROC** 以及 **Precision-Recall 曲线**。
2. **差异化预处理**: 修正 `train.py` 中的 Pipeline，避免对 One-Hot 编码后的分类特征进行 `StandardScaler` 缩放，仅对数值特征进行标准化。
3. **模型升级**: 逻辑回归（LR）在线性空间处理复杂不平衡数据能力有限，考虑回归到梯度提升树（GBT）并结合采样技术（如 SMOTE）。

---
*记录时间: 2025-12-27*